# ECSE 415 Final Project: Deep Learning Classifier (Bonus 10 Points)

Group 10 

April 2022

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

'''
Common google drive for project files and dataset. Mount with this drive.
email: ecse415project2022@gmail.com
password: mcgillecse415
'''

path = "/content/drive/MyDrive/ecse415-project/dataset"

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

import torch
import torchvision
from torchvision import models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Extract images

In [ ]:
all_labels_0000 = np.load(os.path.join(path, 'all_labels_0000.npy'), allow_pickle=True)
all_images_0000 = np.load(os.path.join(path, 'all_images_0000.npy'), allow_pickle=True)

all_labels_0001 = np.load(os.path.join(path, 'all_labels_0001.npy'), allow_pickle=True)
all_images_0001 = np.load(os.path.join(path, 'all_images_0001.npy'), allow_pickle=True)

all_labels_0002 = np.load(os.path.join(path, 'all_labels_0002.npy'), allow_pickle=True)
all_images_0002 = np.load(os.path.join(path, 'all_images_0002.npy'), allow_pickle=True)

all_labels_0003 = np.load(os.path.join(path, 'all_labels_0003.npy'), allow_pickle=True)
all_images_0003 = np.load(os.path.join(path, 'all_images_0003.npy'), allow_pickle=True)

In [ ]:
# Reference: Tutorial 8 - cifar10_tutorial.ipynb 

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

## Dataset and Dataloaders

In [ ]:
class VehiculeDataset(Dataset):
  ''' Vehicule and non-vehicule dataset. '''
  def __init__(self, images, labels, transform=None):
    self.images = images
    self.labels = labels
    self.transform = transform
  
  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    image = self.images[idx]
    label = self.labels[idx]
    if self.transform:
      image = self.transform(image)
    
    return image, label


In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
     transforms.Resize((110, 65))
     ]
)

BATCH_SIZE = 4

# Train
train_images = np.concatenate([all_images_0000, all_images_0001, all_images_0002])
train_labels = np.concatenate([all_labels_0000, all_labels_0001, all_labels_0002])
trainset = VehiculeDataset(train_images, train_labels, transform=transform)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

# Test
test_images = np.concatenate([all_images_0003])
test_labels = np.concatenate([all_labels_0003])
testset = VehiculeDataset(test_images, test_labels, transform=transform)
testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


# Define classes
classes = (0, 1)

## CNN

In [ ]:
# Define devide to use if CUDA is enabled
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
cnn = models.vgg16(pretrained=False, num_classes=len(classes))
print(cnn)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
for param in cnn.parameters():
    param.requires_grad = True

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.001, momentum=0.9)

# Boostrap model to device
cnn.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# Train

NUM_EPOCHS = 8
cnn.train()

# Reference: Tutorial 8 - cifar10_tutorial.ipynb 

for epoch in range(NUM_EPOCHS):
  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):
    # get input and label
    inputs = data[0].to(device)
    labels = data[1]
    labels = labels.type(torch.LongTensor)
    labels = labels.to(device)

    # zero parameter gradients
    optimizer.zero_grad()

    # forward, backward, optimize
    outputs = cnn.forward(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

     # print statistics
    running_loss += loss.item()
    if i % 200 == 199:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 200))
        running_loss = 0.0

print('Training completed.')

[1,   200] loss: 0.695
[1,   400] loss: 0.689
[1,   600] loss: 0.680
[1,   800] loss: 0.676
[1,  1000] loss: 0.672
[1,  1200] loss: 0.638
[1,  1400] loss: 0.644
[1,  1600] loss: 0.636
[1,  1800] loss: 0.611
[1,  2000] loss: 0.571
[1,  2200] loss: 0.520
[1,  2400] loss: 0.551
[1,  2600] loss: 0.522
[1,  2800] loss: 0.499
[1,  3000] loss: 0.542
[1,  3200] loss: 0.468
[1,  3400] loss: 0.464
[1,  3600] loss: 0.477
[1,  3800] loss: 0.484
[1,  4000] loss: 0.447
[1,  4200] loss: 0.451
[1,  4400] loss: 0.429
[1,  4600] loss: 0.460
[1,  4800] loss: 0.452
[1,  5000] loss: 0.381
[1,  5200] loss: 0.411
[1,  5400] loss: 0.421
[1,  5600] loss: 0.368
[1,  5800] loss: 0.425
[1,  6000] loss: 0.372
[1,  6200] loss: 0.364
[1,  6400] loss: 0.422
[1,  6600] loss: 0.346
[1,  6800] loss: 0.382
[1,  7000] loss: 0.375
[1,  7200] loss: 0.365
[1,  7400] loss: 0.369
[1,  7600] loss: 0.331
[1,  7800] loss: 0.313
[1,  8000] loss: 0.361
[1,  8200] loss: 0.375
[1,  8400] loss: 0.362
[1,  8600] loss: 0.315
[1,  8800] 

In [ ]:
# Define state dict file path
saved_model_state_dict_path = '/content/drive/MyDrive/ecse415-project'+'/deep_learning_classifier.pth'

In [ ]:
# Save trained model
torch.save(cnn.state_dict(), saved_model_state_dict_path)

In [ ]:
# Test

correct = 0
total = 0
cnn.eval()

with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        # get input and label
        inputs = data[0].to(device)
        labels = data[1]
        labels = labels.type(torch.LongTensor)
        labels = labels.to(device)
        outputs = cnn.forward(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 89 %


In [ ]:
# To load the model
model = models.vgg16(pretrained=False, num_classes=2)
model.load_state_dict(torch.load(saved_model_state_dict_path, map_location=torch.device(device)))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

# def validation(dloader, val_images, val_labels, model):
#   predictions = np.empty(len(val_images), dtype=float)
#   with torch.no_grad():
#     for i, data in enumerate(dloader, 0):
#         # get input and label
#         inputs = data[0].to(device)
#         labels = data[1]
#         labels = labels.type(torch.LongTensor)
#         labels = labels.to(device)
#         outputs = model.forward(inputs)
#         _, predicted = torch.max(outputs.data, 1)
#         for j, pred in enumerate(predicted, 0):
#           predictions[i+j] = predicted.cpu().detach().numpy()[j]

#     return predictions

# predictions = validation(testloader, all_images_0003, all_labels_0003, model)
# print(predictions)
# acc = np.sum(predictions == all_labels_0003) / len(all_labels_0003)
# precision, recall, _, _ = precision_recall_fscore_support(all_labels_0003, predictions, average='binary')
# cm = confusion_matrix(all_labels_0003, predictions)